In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("../.env-feast")

True

In [2]:
import kfp

# by default, when run from inside a Kubernetes cluster:
#  - the token is read from the `KF_PIPELINES_SA_TOKEN_PATH` path
#  - the host is set to `http://ml-pipeline-ui.kubeflow.svc.cluster.local`
kfp_client = kfp.Client()

# test the client by listing experiments
experiments = kfp_client.list_experiments(namespace="kubeflow-user-example-com")
print(experiments)

{'experiments': [{'created_at': datetime.datetime(2025, 9, 10, 23, 10, 30, tzinfo=tzlocal()),
                  'description': None,
                  'display_name': 'recommender-movie-training-validation',
                  'experiment_id': 'fab88bf8-0642-4c6c-852c-1ead457571d8',
                  'last_run_created_at': datetime.datetime(2025, 9, 11, 0, 29, 16, tzinfo=tzlocal()),
                  'namespace': 'kubeflow-user-example-com',
                  'storage_state': 'AVAILABLE'},
                 {'created_at': datetime.datetime(2025, 9, 11, 0, 35, 26, tzinfo=tzlocal()),
                  'description': None,
                  'display_name': 'movie-recommender-training-validation',
                  'experiment_id': '6e551967-4c67-4f97-a221-e50dcf45198b',
                  'last_run_created_at': datetime.datetime(2025, 9, 11, 1, 13, 31, tzinfo=tzlocal()),
                  'namespace': 'kubeflow-user-example-com',
                  'storage_state': 'AVAILABLE'},
             

/opt/conda/envs/movie-recommender-env/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [3]:
def run_pipeline_from_func(experiment_name: str,
                           run_name: str,
                           pipeline_func: callable,
                           arguments: dict):
    try:
        client = kfp.Client()
        
        run = client.create_run_from_pipeline_func(
            pipeline_func=pipeline_func,
            arguments=arguments,
            experiment_name=experiment_name,
            run_name=run_name
        )
        print(f"Pipeline run started with ID: {run.run_id}")
        return run
    except Exception as e:
        print(f"Error starting pipeline run: {e}")
        return None

# Call the function to run the pipeline
# run = run_pipeline_from_func()

In [2]:
for experiment in experiments.experiments:
    if experiment.display_name not in ('recommender-movie-training-validation', 'default'):
        kfp_client.delete_experiment(experiment.experiment_id)

In [3]:
# Define el nombre del experimento
experiment_name = 'movie-recommender-data-preparation'

# Crea el experimento si no existe
try:
    experiment = kfp_client.create_experiment(name=experiment_name)
    print(f'Experimento "{experiment_name}" creado con éxito.')
except Exception as e:
    print(e)
    # print(f'Error al crear el experimento, probablemente ya existe. Usando el experimento existente.')
    # experiment = client.get_experiment(experiment_name=experiment_name)

Experimento "movie-recommender-data-preparation" creado con éxito.


In [4]:
from data_preparation_pipeline import dataprep_pipeline

experiment_name = 'movie-recommender-data-preparation'

run_pipeline_from_func(experiment_name = experiment_name,
                       run_name='Data Prep Pipeline Run',
                       pipeline_func = dataprep_pipeline,
                       arguments = {'minio_bucket': 'datasets',
                                    'random_init': 42,
                                     'db_name':os.environ["POSTGRES_DB"],
                                     'db_user':os.environ["POSTGRES_USER"],
                                     'db_password':os.environ["POSTGRES_PASSWORD"]})

Pipeline run started with ID: c0e16686-82eb-4094-860d-9f1c5bc330fa


RunPipelineResult(run_id=c0e16686-82eb-4094-860d-9f1c5bc330fa)

In [4]:
# Define el nombre del experimento
experiment_name = 'movie-recommender-training-validation'

# Crea el experimento si no existe
try:
    experiment = kfp_client.create_experiment(name=experiment_name)
    print(f'Experimento "{experiment_name}" creado con éxito.')
except Exception as e:
    print(e)
    # print(f'Error al crear el experimento, probablemente ya existe. Usando el experimento existente.')
    # experiment = client.get_experiment(experiment_name=experiment_name)

Experimento "movie-recommender-training-validation" creado con éxito.


In [5]:
from training_and_validation import training_pipeline

experiment_name = 'movie-recommender-training-validation'

run_pipeline_from_func(experiment_name = experiment_name,
                       run_name='Training and validation',
                       pipeline_func = training_pipeline,
                       arguments = {})

Pipeline run started with ID: 11d8cee6-d53b-44d8-b882-4c4ef568f10f


RunPipelineResult(run_id=11d8cee6-d53b-44d8-b882-4c4ef568f10f)

In [3]:
# Define el nombre del experimento
experiment_name = 'movie-recommender-training-validation-cuda'

# Crea el experimento si no existe
try:
    experiment = kfp_client.create_experiment(name=experiment_name)
    print(f'Experimento "{experiment_name}" creado con éxito.')
except Exception as e:
    print(e)
    # print(f'Error al crear el experimento, probablemente ya existe. Usando el experimento existente.')
    # experiment = client.get_experiment(experiment_name=experiment_name)

Experimento "movie-recommender-training-validation-cuda" creado con éxito.


In [4]:
from training_and_validation_cuda import training_pipeline_cuda

run_pipeline_from_func(experiment_name = experiment_name,
                       run_name='Training and validation',
                       pipeline_func = training_pipeline_cuda,
                       arguments = {"training_epochs": 1})

/opt/conda/envs/movie-recommender-env/lib/python3.11/site-packages/kfp/dsl/component_decorator.py:126: FutureWarning: The default base_image used by the @dsl.component decorator will switch from 'python:3.9' to 'python:3.10' on Oct 1, 2025. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.10.
  return component_factory.create_component_from_func(


Pipeline run started with ID: 251d7ac9-1d06-4a3f-b263-c9a0a588136c


RunPipelineResult(run_id=251d7ac9-1d06-4a3f-b263-c9a0a588136c)

# 